In [25]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Callable, Optional, Tuple, Iterable

import warnings
import numpy as np
import torch
from torch import Tensor
from torch.distributions import Normal
from torch.quasirandom import SobolEngine

from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.acquisition import AcquisitionFunction
from botorch.optim import optimize_acqf
from botorch.utils.transforms import normalize, unnormalize
from botorch.models.utils.gpytorch_modules import get_covar_module_with_dim_scaled_prior
from botorch.models.ensemble import EnsembleModel
from botorch.posteriors.ensemble import EnsemblePosterior

from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.settings import cholesky_jitter

from sklearn.ensemble import RandomForestRegressor

# warnings
from linear_operator.utils.warnings import NumericalWarning
from botorch.exceptions.warnings import NumericsWarning
from botorch.exceptions import BadInitialCandidatesWarning
import time
import os

In [26]:
# core code for bayesian optimisation
def snap(x: float | Tensor, choices: np.ndarray | Tensor) -> Tensor:
    if not torch.is_tensor(choices):
        choices = torch.tensor(choices, dtype=torch.double)
    x = torch.as_tensor(x, dtype=torch.double)
    idx = torch.argmin(torch.abs(choices - x))
    return choices[idx]


class ExpectedImprovement(AcquisitionFunction):
    def __init__(self, model, best_f: float, xi: float = 0.01, maximize: bool = True):
        super().__init__(model)
        self.best_f = float(best_f)
        self.xi = float(xi)
        self.maximize = maximize
        self._normal = Normal(0, 1)

    def forward(self, X: Tensor) -> Tensor:
        post = self.model.posterior(X)
        mean = post.mean.squeeze(-1)
        std = post.variance.clamp_min(1e-9).sqrt().squeeze(-1)
        if self.maximize:
            u = (mean - self.best_f - self.xi) / std
            ei = (mean - self.best_f - self.xi) * self._normal.cdf(u) + std * self._normal.log_prob(u).exp()
        else:
            u = (self.best_f - mean - self.xi) / std
            ei = (self.best_f - mean - self.xi) * self._normal.cdf(u) + std * self._normal.log_prob(u).exp()
        ei = ei.squeeze(-1)
        if ei.ndim > 1:
            raise RuntimeError(f"Expected 1D tensor, got shape {ei.shape}")
        return ei


class EnsembleRandomForestModel(EnsembleModel):
    model: RandomForestRegressor
    num_samples: int
    _num_outputs: int

    def __init__(self, num_samples: int = 100, random_state: int = 0):
        super().__init__()
        self._num_outputs = 1
        self.model = RandomForestRegressor(
            n_estimators=num_samples, random_state=random_state, n_jobs=-1
        )

    def fit(self, X: Tensor, y: Tensor) -> None:
        X_np = X.detach().cpu().numpy()
        y_np = y.detach().cpu().numpy().reshape(-1)
        self.model.fit(X_np, y_np)

    def forward(self, X: Tensor) -> Tensor:
        device, dtype = X.device, X.dtype
        *batch_shape, q, d = X.shape
        X_np = X.detach().cpu().reshape(-1, d).numpy()

        preds = np.stack([est.predict(X_np) for est in self.model.estimators_], axis=0)

        preds_t = torch.from_numpy(preds).to(device=device, dtype=dtype)
        preds_t = preds_t.view(len(self.model.estimators_), *batch_shape, q)
        preds_t = preds_t.permute(*range(1, preds_t.dim()-1), 0, preds_t.dim()-1)
        preds_t = preds_t.unsqueeze(-1)

        return preds_t

    def posterior(self, X: Tensor, **kwargs) -> EnsemblePosterior:
        samples = self.forward(X)
        return EnsemblePosterior(samples)


@dataclass
class MagneticBayesianOptimizationConfig:
    objective: Callable[[Tensor], Tensor]
    ku: np.ndarray
    damp: np.ndarray
    bounds_real: Tensor

    # model choice: "gp" or "rf"
    model_type: str = "gp"

    # GP kernel (used only when model_type="gp")
    kernel: str = "matern"

    # RF options (used only when model_type="rf")
    rf_n_estimators: int = 100
    rf_random_state: Optional[int] = None

    # loop / behavior
    n_initial: int = 500
    n_runs: int = 150

    # acquisition choice
    acq: str = "ei"
    xi: float = 0.1
    neighbor_xi: float = 0.01

    # NAS / SDR
    use_nas: bool = True
    use_sdr: bool = True
    shrink_factor: Tensor = torch.tensor([0.9, 0.945], dtype=torch.double)
    min_window: float = 0.0
    filter_outside_window: bool = True

    # acquisition optimizer
    raw_samples: int = 500
    num_restarts: int = 10

    # reproducibility & statistics
    seed: Optional[int] = None
    suppress_warnings: bool = True
    verbose: bool = False
    print_total_time: bool = True


def Sequential_Domain_Reduction(
    best_point: torch.Tensor,
    prev_point: Optional[torch.Tensor],
    current_bounds: torch.Tensor,
    limits: torch.Tensor,
    prev_d: Optional[torch.Tensor] = None,
    gamma_osc: float = 0.7,
    gamma_pan: float = 1.0,
    eta: float | torch.Tensor = 0.9,
    min_window: float | torch.Tensor = 0.05,
):
    d = best_point.shape[0]
    lower, upper = current_bounds
    lim_lower, lim_upper = limits
    device, dtype = best_point.device, best_point.dtype

    if not torch.is_tensor(eta):
        eta = torch.full((d,), float(eta), dtype=dtype, device=device)
    else:
        eta = eta.to(device=device, dtype=dtype)
    if not torch.is_tensor(min_window):
        min_window = torch.full((d,), float(min_window), dtype=dtype, device=device)
    else:
        min_window = min_window.to(device=device, dtype=dtype)

    r_prev = (upper - lower).clamp_min(torch.finfo(dtype).eps)
    if prev_point is None:
        prev_point = best_point

    curr_d = 2.0 * (best_point - prev_point) / r_prev
    if prev_d is None:
        prev_d = torch.zeros_like(curr_d)

    c = curr_d * prev_d
    c_hat = torch.sign(c) * torch.sqrt(torch.abs(c))

    gamma = 0.5 * (gamma_pan * (1.0 + c_hat) + gamma_osc * (1.0 - c_hat))
    lam = (eta + torch.abs(curr_d) * (gamma - eta)).clamp(0.0, 1.0)

    r_new = lam * r_prev
    center = best_point

    half = r_new / 2.0
    new_lower = center - half
    new_upper = center + half

    for i in range(d):
        if new_lower[i] < lim_lower[i]:
            shift = lim_lower[i] - new_lower[i]
            new_lower[i] += shift
            new_upper[i] += shift
        elif new_upper[i] > lim_upper[i]:
            shift = new_upper[i] - lim_upper[i]
            new_lower[i] -= shift
            new_upper[i] -= shift

    for i in range(d):
        width = new_upper[i] - new_lower[i]
        if width < min_window[i]:
            mid = 0.5 * (new_lower[i] + new_upper[i])
            half_min = 0.5 * min_window[i]
            new_lower[i] = torch.max(lim_lower[i], mid - half_min)
            new_upper[i] = torch.min(lim_upper[i], mid + half_min)

    return new_lower, new_upper, curr_d.detach()


# -------- runner --------

class MagneticBayesianOptimization:
    
    def __init__(self, cfg: MagneticBayesianOptimizationConfig):
        self.cfg = cfg
        self.seed = self._init_global_seed(cfg.seed)

        if self.cfg.suppress_warnings:
            warnings.filterwarnings("ignore", category=BadInitialCandidatesWarning)
            warnings.filterwarnings("ignore", category=NumericalWarning)
            warnings.filterwarnings("ignore", category=NumericsWarning)
            warnings.filterwarnings("ignore", category=RuntimeWarning, message="Optimization failed *")

    @staticmethod
    def _init_global_seed(seed: Optional[int]) -> int:
        import secrets as _secrets
        if seed is None:
            seed = _secrets.randbits(32)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)
        return seed

    @staticmethod
    def get_neighbors_2d(point: Tuple[float, float], ku_vals: Iterable[float], damp_vals: Iterable[float]):
        ku_vals = np.asarray(ku_vals); damp_vals = np.asarray(damp_vals)
        ku, damp = point
        i_arr = np.where(ku_vals == ku)[0]
        j_arr = np.where(damp_vals == damp)[0]
        if len(i_arr) == 0 or len(j_arr) == 0:
            return []
        i, j = int(i_arr[0]), int(j_arr[0])
        nbrs = []
        for di, dj in [(-1,0),(1,0),(0,-1),(0,1)]:
            ni, nj = i+di, j+dj
            if 0 <= ni < len(ku_vals) and 0 <= nj < len(damp_vals):
                nbrs.append((float(ku_vals[ni]), float(damp_vals[nj])))
        return nbrs

    def _initial_qmc(self, n: int, bounds_real: Tensor):
        sobol = SobolEngine(dimension=2, scramble=True, seed=self.seed)
        U = sobol.draw(n).double()
        X_real = U * (bounds_real[1] - bounds_real[0]) + bounds_real[0]
        y_vals = [self.cfg.objective(x) for x in X_real]
        Y = torch.as_tensor([[float(y.item())] for y in y_vals], dtype=torch.double)
        return X_real, Y

    def _build_gp(self, Xn: Tensor, Y: Tensor) -> SingleTaskGP:
        train_Yvar = torch.full_like(Y, 1e-6)
        use_rbf = (self.cfg.kernel.lower() == "rbf")
        covar = get_covar_module_with_dim_scaled_prior(
            ard_num_dims=Xn.shape[-1],
            batch_shape=Xn.shape[:-2],
            use_rbf_kernel=use_rbf,  # False => Matern (default) via helper
        )
        gp = SingleTaskGP(train_X=Xn, train_Y=Y, train_Yvar=train_Yvar, covar_module=covar)
        mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
        try:
            fit_gpytorch_mll(mll)
        except Exception:
            with cholesky_jitter(1e-3):
                fit_gpytorch_mll(mll)
        return gp

    def _build_rf(self, Xn: Tensor, Y: Tensor) -> EnsembleRandomForestModel:
        rs = self.cfg.rf_random_state if self.cfg.rf_random_state is not None else self.seed
        model = EnsembleRandomForestModel(num_samples=self.cfg.rf_n_estimators, random_state=int(rs))
        model.fit(Xn, Y)
        return model

    def _make_model(self, Xn: Tensor, Y: Tensor):
        if self.cfg.model_type.lower() == "rf":
            return self._build_rf(Xn, Y)
        # default: GP
        return self._build_gp(Xn, Y)

    def _make_acq(self, model, best_Y: float, *, for_neighbors: bool = False):
        
        xi = self.cfg.neighbor_xi if for_neighbors else self.cfg.xi
        return ExpectedImprovement(model=model, best_f=best_Y, xi=xi, maximize=True)

    def _propose(self, model, unit_bounds: Tensor, best_Y: float) -> Tensor:
        acq = self._make_acq(model, best_Y, for_neighbors=False)
        
        opts = {"sample_around_best": True, "seed": self.seed}
        if self.cfg.model_type.lower() == "rf":
            opts["with_grad"] = False
        cands, _ = optimize_acqf(
            acq_function=acq,
            bounds=unit_bounds,
            q=1,
            num_restarts=self.cfg.num_restarts,
            raw_samples=self.cfg.raw_samples,
            return_best_only=True,
            options=opts,
        )
        return cands

    def _neighbor_aware_update(
        self,
        unnormalized_next: Tensor,
        bounds_real: Tensor,
        model,
        Xn: Tensor,
        Y: Tensor,
        seen_points: set[tuple[float, float]],
    ) -> Tuple[Tensor, Tensor]:
        best_point_real: Optional[Tensor] = None
        best_key: Optional[tuple[float, float]] = None
        best_val: float = float("-inf")

        for cand in unnormalized_next:
            cn = cand.detach().cpu().numpy()
            snapped_tuple = (float(snap(cn[0], self.cfg.ku)), float(snap(cn[1], self.cfg.damp)))

            if snapped_tuple not in seen_points:
                val = float(self.cfg.objective(torch.tensor(snapped_tuple, dtype=torch.double)).item())
                if val > best_val:
                    best_val = val
                    best_point_real = cand
                else:
                    seen_points.add(snapped_tuple)

            neighbors = [nb for nb in self.get_neighbors_2d(snapped_tuple, self.cfg.ku, self.cfg.damp)
                         if nb not in seen_points]
            if neighbors:
                neighbor_tensor = torch.tensor(neighbors, dtype=torch.double)
                normalized_neighbors = normalize(neighbor_tensor, bounds_real)

                with torch.no_grad():
                    ei_acq = self._make_acq(model, best_Y=float(Y.max().item()), for_neighbors=True)
                    ei = ei_acq(normalized_neighbors).squeeze(-1)

                top_idx = int(torch.argmax(ei).item())
                best_nb = neighbors[top_idx]
                val_nb = float(self.cfg.objective(torch.tensor(best_nb, dtype=torch.double)).item())

                if val_nb > best_val:
                    best_val = val_nb
                    best_point_real = torch.tensor(best_nb, dtype=torch.double)
                    best_key = best_nb
                else:
                    seen_points.add(best_nb)

        if best_point_real is not None:
            if best_key is not None:
                seen_points.add(best_key)
            new_Xn = normalize(best_point_real.unsqueeze(0), bounds_real)
            new_Y = torch.tensor([[best_val]], dtype=torch.double)
            Xn = torch.cat([Xn, new_Xn], dim=0)
            Y = torch.cat([Y, new_Y], dim=0)

        return Xn, Y

    @staticmethod
    def _fmt_bounds(bounds_real: Tensor) -> str:
        lower, upper = bounds_real[0], bounds_real[1]
        return (f"[{float(lower[0]):.3f},{float(upper[0]):.3f}] x "
                f"[{float(lower[1]):.3f},{float(upper[1]):.3f}]")

    def run(self) -> Tuple[float, float]:
        """Run BO and return ONLY (ku_opt, damp_opt). Fresh seen_points per run."""
        t0 = time.perf_counter()
        bounds_real = self.cfg.bounds_real.to(dtype=torch.double)
        seen_points: set[tuple[float, float]] = set()

        
        X0_real, Y = self._initial_qmc(self.cfg.n_initial, bounds_real)
        Xn = normalize(X0_real, bounds_real)

        current_bounds = bounds_real.clone()
        unit_bounds = torch.tensor([[0.0, 0.0], [1.0, 1.0]], dtype=torch.double)
        prev_point = None
        prev_d = None

        for it in range(self.cfg.n_runs):
            model = self._make_model(Xn, Y)
            cands_n = self._propose(model, unit_bounds, best_Y=float(Y.max().item()))
            cands_real = unnormalize(cands_n, bounds_real)

            if self.cfg.use_nas:
                Xn, Y = self._neighbor_aware_update(
                    cands_real, bounds_real, model, Xn, Y, seen_points
                )
            else:
                
                cand = cands_real[0]
                val = float(self.cfg.objective(cand).item())
                Xn = torch.cat([Xn, cands_n[0].unsqueeze(0)], dim=0)
                Y = torch.cat([Y, torch.tensor([[val]], dtype=torch.double)], dim=0)

            
            best_idx = torch.argmax(Y)
            best_point_real = unnormalize(Xn[best_idx], bounds_real)
            best_y = float(Y[best_idx].item())

            if self.cfg.use_sdr:
                new_lower, new_upper, prev_d = Sequential_Domain_Reduction(
                    best_point=best_point_real,
                    prev_point=prev_point,
                    current_bounds=current_bounds,
                    limits=bounds_real,
                    prev_d=prev_d,
                    eta=self.cfg.shrink_factor,
                    min_window=self.cfg.min_window,
                )
                prev_point = best_point_real
                current_bounds[0] = new_lower
                current_bounds[1] = new_upper
                unit_bounds = normalize(current_bounds, bounds_real)

                if self.cfg.filter_outside_window:
                    if self.cfg.model_type.lower() == "rf" and it==0:
                        warnings.warn(
                            f"RF: filtered training point(s) outside SDR window. "
                            "This changes the RF training set and may affect performance and reproducibility.",
                            UserWarning,
                        )

                    mask = (
                        (Xn[:, 0] >= unit_bounds[0, 0]) & (Xn[:, 0] <= unit_bounds[1, 0]) &
                        (Xn[:, 1] >= unit_bounds[0, 1]) & (Xn[:, 1] <= unit_bounds[1, 1])
                    )
                    keep = torch.nonzero(mask, as_tuple=False).squeeze(-1)
                    if keep.numel() >= 1:
                        Xn = Xn[keep]
                        Y = Y[keep]

            if self.cfg.verbose:
                ku_best = float(snap(best_point_real[0].item(), self.cfg.ku))
                damp_best = float(snap(best_point_real[1].item(), self.cfg.damp))
                print(
                    f"[iter {it+1}/{self.cfg.n_runs}] "
                    f"best_Y={best_y:.6g}, best=(ku={ku_best:.3f}, damp={damp_best:.3f}), "
                    f"domain={self._fmt_bounds(current_bounds)}, model={self.cfg.model_type}"
                )

        best_idx = torch.argmax(Y)
        best_params_real = unnormalize(Xn[best_idx].unsqueeze(0), bounds_real).squeeze(0)
        ku_opt = float(snap(best_params_real[0].item(), self.cfg.ku))
        damp_opt = float(snap(best_params_real[1].item(), self.cfg.damp))

        if self.cfg.print_total_time:
            elapsed = time.perf_counter() - t0
            print(f"[run] total time: {elapsed:.3f}s")

        return ku_opt, damp_opt

Normal and Low Frequency Fast Fourier Transform objective functions

In [27]:
def objective_func(candidate: torch.Tensor)  -> torch.Tensor:
  anis = candidate[0].item()
  damp = candidate[1].item()

  anis = snap(anis, Ku)
  damp = snap(damp, Damp)

  anis = format(anis, '.3f')
  damp = format(damp, '.3f')

  filename = os.path.join(source, f'Anis_{anis}e-22Damp_{damp}', 'output')
  
  W0 = torch.tensor(np.loadtxt(filename), dtype=torch.double)

  chi_all = torch.sum((W0[6:, 2:5] - Wc[6:, 2:5]) ** 2)

  return torch.neg(chi_all)

def objective_func_fft(candidate):
    anis = snap(candidate[0], Ku)
    damp = snap(candidate[1], Damp)

    anis_str = format(anis, '.3f')
    damp_str = format(damp, '.3f')

    filename = os.path.join(source, f'Anis_{anis_str}e-22Damp_{damp_str}', 'output')
    W0 = torch.tensor(np.loadtxt(filename), dtype=torch.double)

    W0_sig = W0[6:, 2:5]

    Wc_sig = Wc[6:, 2:5]

    alpha = 1.0

    beta = 6.073e-3
    N_freq = 9

    value_loss = torch.sum((W0_sig - Wc_sig) ** 2)

    W0_fft = torch.fft.rfft(W0_sig, dim=0)
    Wc_fft = torch.fft.rfft(Wc_sig, dim=0)
    W0_fft_low = W0_fft[:N_freq, :]
    Wc_fft_low = Wc_fft[:N_freq, :]
    fft_loss = torch.sum(torch.view_as_real(W0_fft_low - Wc_fft_low) ** 2)

    total_loss = alpha * value_loss + beta * fft_loss 

    return -total_loss

Define relative file paths here

In [ ]:
head = %pwd


root = head + '/Desktop/University/Postgrad/Masters_Project/ZendTo-7VvUZDh2zSQT44Rd/grid_search_method'

big_data_set_file_path = root + '/big_dataset/single_spin_reference_nofield_30_new'
ensemble_data_file_path = root + '/real_data'

Define Anisotropy Damping Ranges, Step Sizes and intial bounds of the search space

In [29]:
Ku = np.arange(0.0, 2.7, 0.015)
Damp = np.arange(0.01, 0.2001, 0.001)
bounds_real = torch.tensor([[Ku.min(), Damp.min()],
                            [Ku.max(), Damp.max()]], dtype=torch.double)

BO + Gaussian Process + Sequential Domain Reduction + Neighbour Aware Search + Expected Improvement $\xi=0.1$
    + Matern 5/2 + Low Frequency Fast Fourier Transform Loss

In [20]:
# Uncomment and Modify this file path to test ensemble_data at elevated temperatures
# file_comp_botorch =  ensemble_data_file_path + '/anis_2.63e-22/output_av_temp500.1'

# Uncomment and Modify this file path to test single-spin data in the big data set
anisotropy = '2.625'
damping = '0.051'
file_comp_botorch = big_data_set_file_path + f'/Anis_{anisotropy}e-22Damp_{damping}/output'

Wc = torch.tensor(np.loadtxt(file_comp_botorch), dtype=torch.double)
cfg = MagneticBayesianOptimizationConfig(
    objective=objective_func_fft,
    ku=Ku,
    damp=Damp,
    bounds_real=bounds_real,
    n_initial=500,
    n_runs=150,
    xi=0.1,
    use_sdr=True,
    use_nas=True,
    acq="ei",
    kernel="gp",          
    shrink_factor=torch.tensor([0.9, 0.945], dtype=torch.double),
    raw_samples=500,
    num_restarts=10,
    filter_outside_window=True,
    verbose=True,
    seed=None,      
)

runner = MagneticBayesianOptimization(cfg)
ku_opt, damp_opt = runner.run()
print(f"Extracted Anisotropy Pairs: {ku_opt}, {damp_opt}")

[iter 1/150] best_Y=-0.221783, best=(ku=2.655, damp=0.050), domain=[0.270,2.700] x [0.010,0.190], model=gp
[iter 2/150] best_Y=-0.221783, best=(ku=2.655, damp=0.050), domain=[0.513,2.700] x [0.010,0.180], model=gp
[iter 3/150] best_Y=-0.0299857, best=(ku=2.625, damp=0.048), domain=[0.735,2.700] x [0.010,0.170], model=gp
[iter 4/150] best_Y=-0.0299857, best=(ku=2.625, damp=0.048), domain=[0.931,2.700] x [0.010,0.161], model=gp
[iter 5/150] best_Y=-0.0299857, best=(ku=2.625, damp=0.048), domain=[1.108,2.700] x [0.010,0.153], model=gp
[iter 6/150] best_Y=-0, best=(ku=2.625, damp=0.051), domain=[1.267,2.700] x [0.010,0.144], model=gp
[iter 7/150] best_Y=-0, best=(ku=2.625, damp=0.051), domain=[1.411,2.700] x [0.010,0.137], model=gp
[iter 8/150] best_Y=-0, best=(ku=2.625, damp=0.051), domain=[1.540,2.700] x [0.010,0.130], model=gp
[iter 9/150] best_Y=-0, best=(ku=2.625, damp=0.051), domain=[1.656,2.700] x [0.010,0.123], model=gp
[iter 10/150] best_Y=-0, best=(ku=2.625, damp=0.051), domain=[

Random Forest

In [23]:
anisotropy = '2.625'
damping = '0.051'

file_comp_botorch = big_data_set_file_path + f'/Anis_{anisotropy}e-22Damp_{damping}/output'
Wc = torch.tensor(np.loadtxt(file_comp_botorch), dtype=torch.double)
cfg = MagneticBayesianOptimizationConfig(
    objective=objective_func,
    ku=Ku, 
    damp=Damp, 
    bounds_real=bounds_real,
    model_type="rf",                
    rf_n_estimators=10,            
    rf_random_state=None,           
    acq="ei",                
    filter_outside_window=False,
    n_initial=500,
    n_runs=150,
    xi=0.1,
    use_nas=True, 
    use_sdr=True,
    kernel="matern",                
    verbose=True,                   
    seed=None,                       
)

runner = MagneticBayesianOptimization(cfg)
ku_opt, damp_opt = runner.run()
print(f"Extracted Anisotropy Pairs: {ku_opt}, {damp_opt}")

[iter 1/150] best_Y=-0.0509057, best=(ku=2.640, damp=0.047), domain=[0.270,2.700] x [0.010,0.190], model=rf
[iter 2/150] best_Y=-0.0509057, best=(ku=2.640, damp=0.047), domain=[0.513,2.700] x [0.010,0.180], model=rf
[iter 3/150] best_Y=-0.0509057, best=(ku=2.640, damp=0.047), domain=[0.732,2.700] x [0.010,0.170], model=rf
[iter 4/150] best_Y=-0.0509057, best=(ku=2.640, damp=0.047), domain=[0.929,2.700] x [0.010,0.162], model=rf
[iter 5/150] best_Y=-0.0509057, best=(ku=2.640, damp=0.047), domain=[1.106,2.700] x [0.010,0.153], model=rf
[iter 6/150] best_Y=-0.0509057, best=(ku=2.640, damp=0.047), domain=[1.265,2.700] x [0.010,0.145], model=rf
[iter 7/150] best_Y=-0.0509057, best=(ku=2.640, damp=0.047), domain=[1.409,2.700] x [0.010,0.138], model=rf
[iter 8/150] best_Y=-0.0509057, best=(ku=2.640, damp=0.047), domain=[1.538,2.700] x [0.010,0.131], model=rf
[iter 9/150] best_Y=-0.0509057, best=(ku=2.640, damp=0.047), domain=[1.654,2.700] x [0.010,0.124], model=rf
[iter 10/150] best_Y=-0.0509